In [2]:
from pydantic import BaseModel
from typing import Optional
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS
from pyfuseki import FusekiUpdate
from rdflib.plugins.stores import sparqlstore
from rdflib.namespace import XSD
from datetime import datetime
from src.schemas.settings import Settings

In [3]:
class BfElement(BaseModel):
    value: str
    label: str

class ResourceProperties(BaseModel):
    content: BfElement
    genreForm: Optional[BfElement] = None

class AdminMetadata(BaseModel):
    creationDate: str
    status: BfElement
    descriptionConventions: BfElement
    identifiedBy: str

class Title(BaseModel):
    mainTitle: str
    subtitle: Optional[str] = None

class Contribution(BaseModel):
    term: BfElement
    role: BfElement

class Subject(BaseModel):
    type: str
    lang: str
    term: BfElement

class Classification(BaseModel):
    cdd: str
    cutter: str

class BfWork(BaseModel):
    resourceType: list[BfElement]
    adminMetadata: AdminMetadata
    title: Title
    variantTitle: Optional[list[Title]] = None
    language: list[BfElement]
    genreForm: list[BfElement] = None
    contribution: Optional[list[Contribution]] = None
    subject: Optional[list[Subject]] = None
    classification: Classification



In [6]:
req = {
    "resourceType": [
        {
            "value": "Work",
            "label": "Obra"
        },
        {
            "value": "Text",
            "label": "Texto"
        }
    ],
    "adminMetadata": {
        "creationDate": "08/03/2024",
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        },
        "identifiedBy": "1"
    },
    "title": {
        "mainTitle": "Titulo principal",
        "subtitle": "subtítulo"
    },
    "variantTitle": [
        {
            "mainTitle": "Título alternativo",
            "subtitle": ""
        }
    ],
    "language": [
        {
            "value": "por",
            "label": "Português"
        }
    ],
    "genreForm": [
        {
            "value": "http://id.loc.gov/authorities/genreForms/gf2014026074",
            "label": "Não ficção criativa"
        }
    ],
    "contribution": [
        {
            "term": {
                "value": "https://bibliokeia.com/authority/PersonalName/10",
                "label": "Popper, Karl R."
            },
            "role": {
                "value": "http://id.loc.gov/vocabulary/relators/aut",
                "label": "Autor"
            }
        }
    ],
    "subject": [
        {
            "type": "Topic",
            "term": {
                "value": "https://bibliokeia.com/authority/Topic/11",
                "label": "Metodologia"
            },
            "lang": "por",
            "base": "bk",
            "label": "Termo Tópico"
        }
    ],
    "classification": {
        "cdd": "548.5",
        "cutter": "O48s"
    }
}
request = BfWork(**req)

In [5]:
settings = Settings()

In [21]:
def BfType(g, resource, resourceType):
    for i in resourceType:
        bfType = URIRef(f"http://id.loc.gov/ontologies/bibframe/{i.value}")
        g.add((resource, RDF.type, bfType))
    return g

def BfGenreForm(g, resource, genreForm, BF):
    for i in genreForm:
        uri = URIRef(i.value)
        g.add((uri, RDF.type, BF.GenreForm ))
        g.add((uri, RDFS.label, Literal(i.label)))
        g.add((resource, BF.genreForm, uri))
    return g


def BfAdminMetadata(g, resource, BF):
    now = datetime.now()  

    adminMetadata = BNode()
    g.add((resource, BF.adminMetadata, adminMetadata))
    g.add((adminMetadata, RDF.type, BF.AdminMetadata ))
    # assigner
    assigner_uri = URIRef(settings.organization_loc_uri)
    g.add((assigner_uri, RDF.type, BF.Organization))
    g.add((assigner_uri, RDFS.label, Literal(settings.organization)))
    g.add((adminMetadata, BF.assigner, assigner_uri ))
    # creationDate
    formatted_date = now.strftime("%Y-%m-%d")
    g.add((adminMetadata, BF.creationDate, Literal(formatted_date, datatype=XSD.date) ))
    # descriptionConventions
    descriptionConventions = URIRef(request.adminMetadata.descriptionConventions.value)
    g.add((descriptionConventions, RDF.type, BF.DescriptionConventions ))
    g.add((descriptionConventions, RDFS.label, Literal(request.adminMetadata.descriptionConventions.label) ))
    g.add((adminMetadata, BF.descriptionConventions, descriptionConventions ))
    # generationProcess
    formatted_dateTime = now.strftime("%Y-%m-%dT%H:%M:%S")
    generationProcess = BNode() 
    g.add((adminMetadata, BF.generationProcess, generationProcess))
    g.add((generationProcess, RDF.type, BF.GenerationProcess))
    g.add((generationProcess, RDFS.label, Literal(settings.app_name) ))
    g.add((generationProcess, BF.generationDate, Literal(formatted_dateTime, datatype=XSD.dateTime) ))
    # identifiedBy
    identifiedBy =  BNode() 
    g.add((adminMetadata, BF.identifiedBy, identifiedBy))
    g.add((identifiedBy, RDF.type, BF.Local))
    g.add((identifiedBy, BF.assigner, assigner_uri))
    g.add((identifiedBy, RDF.value, Literal(request.adminMetadata.identifiedBy)))
    # status
    status = URIRef(request.adminMetadata.status.value)
    g.add((status, RDF.type, BF.Status))
    g.add((status, RDFS.label, Literal(request.adminMetadata.status.label)))
    g.add((adminMetadata, BF.status, status))

    return g

def BfTitle(g, resource, BF):
    title = BNode()
    g.add((resource, BF.title, title))
    g.add((title, RDF.type, BF.Title))
    g.add((title, BF.mainTitle, Literal(request.title.mainTitle)))
    if request.title.subtitle:
        g.add((title, BF.subtitle, Literal(request.title.subtitle)))
    if request.variantTitle:
        for i in request.variantTitle:
            variantTitle = BNode()
            g.add((resource, BF.title, variantTitle))
            g.add((variantTitle, RDF.type, BF.VariantTitle))
            g.add((variantTitle, BF.mainTitle, Literal(i.mainTitle)))
            if i.subtitle:
                g.add((variantTitle, BF.subtitle, Literal(i.subtitle)))

    return g

# def BFResourceProperties(g, resource, BF):
#     content = URIRef(request.resourceProperties.content.value)
#     g.add((content, RDF.type, BF.Content))
#     g.add((content, RDFS.label, Literal(request.resourceProperties.content.label)))
#     g.add((resource, BF.content, content))
#     if request.resourceProperties.genreForm:
#         genreForm = URIRef(request.resourceProperties.genreForm.value)
#         g.add((genreForm, RDF.type, BF.GenreForm ))
#         g.add((genreForm, RDFS.label, Literal(request.resourceProperties.genreForm.label)))
#         g.add((resource, BF.genreForm, content))

#     return g

def BFLanguage(g, resource, BF):
    for i in request.language:
        language = URIRef(f"http://id.loc.gov/vocabulary/languages/{i.value}")
        g.add((language, RDF.type, BF.Language))
        g.add((language, RDFS.label, Literal(i.label)))
        g.add((resource, BF.language, language ))
    return g  

def BFContribution(g, resource, BF):
    for i in request.contribution:
        contribution = BNode()
        g.add((resource, BF.contribution, contribution))
        g.add((contribution, RDF.type, BF.Contribution))
        agent = URIRef(i.term.value)
        g.add((agent, RDF.type, BF.Agent))
        g.add((agent, RDF.type, BF.Person))
        g.add((agent, RDFS.label, Literal(i.term.label)))
        g.add((contribution, BF.agent, agent))
        role = URIRef(i.role.value)
        g.add((role, RDF.type, BF.Role))
        g.add((role, RDFS.label, Literal(i.role.label)))
        g.add((contribution, BF.role, role))

    return g

def BFSubject(g, resource, BF):
    for i in request.subject:
        subject = URIRef(i.term.value)
        g.add((subject, RDF.type, BF.Topic))
        g.add((subject, RDFS.label, Literal(i.term.label, lang=i.lang)))
        g.add((resource, BF.subject, subject))

    return g

def BfClassification(g, resource, classification, BF):
    cdd = BNode()
    g.add((resource, BF.classification, cdd))
    g.add((cdd, RDF.type, BF.ClassificationDdc))
    g.add((cdd, BF.classificationPortion, Literal(classification.cdd)))
    cutter = BNode()
    g.add((resource, BF.classification, cutter))
    g.add((cutter, RDF.type, BF.Classification))
    g.add((cutter, BF.classificationPortion, Literal(classification.cutter)))
    return g



In [15]:
request.classification

Classification(cdd='548.5', cutter='O48s')

In [22]:
def MakeGraphWork(request):

    BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
    identifier = f"https://bibliokeia/works/{request.adminMetadata.identifiedBy}"
    resource = URIRef(identifier)

    g = Graph(identifier=identifier)
    g.bind('bf', BF)
    
    # resourceType
    BfType(g, resource, request.resourceType)
    # AdminMetadata
    g = BfAdminMetadata(g, resource, BF) 
    # BfClassification
    g = BfClassification(g, resource, request.classification, BF)
    # Title
    g = BfTitle(g, resource, BF)
    # Language
    g = BFLanguage(g, resource, BF)
    # Contribution
    g = BFContribution(g, resource, BF)
    # Subject
    g = BFSubject(g, resource, BF) 
    # GenreForm
    if request.genreForm:
        g = BfGenreForm(g, resource, request.genreForm, BF)    
 
    graph = g.serialize(format='ttl')
    g.serialize(destination="work_test.ttl", format='ttl')

    return graph

def MakeSparql(graph):
    graph = graph.split("\n\n")
    graph = "\n\n".join(graph[1:])

    graph = f"""
        PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        INSERT DATA {{
            GRAPH <https://bibliokeia/works/10> 
            {{
                {graph} }} }} """

    return graph



graph = MakeGraphWork(request)
sparql = MakeSparql(graph)
print(sparql)


        PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        INSERT DATA {
            GRAPH <https://bibliokeia/works/10> 
            {
                <https://bibliokeia/works/1> a bf:Text,
        bf:Work ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            bf:creationDate "2024-03-12"^^xsd:date ;
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/aacr> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "BiblioKeia" ;
                    bf:generationDate "2024-03-12T15:21:40"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
   

In [39]:
fuseki = FusekiUpdate("http://localhost:3030" , 'bk') 
response = fuseki.run_sparql(sparql) 
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [48]:
update_endpoint = 'http://localhost:3030/bk/update'
query_endpoint = 'http://localhost:3030/bk/query'

store = sparqlstore.SPARQLUpdateStore(query_endpoint=query_endpoint, update_endpoint=update_endpoint)
# store.open((query_endpoint, update_endpoint))

In [50]:
# url = 'http://localhost:3030/bk/data?graph=https:%2F%2Fbibliokeia%2Fworks%2F10'
url = "http://id.loc.gov/resources/works/21984730.rdf"
g = Graph()
g.parse(url)

<Graph identifier=Nf8a4447e118c409ab437162e76a2afcd (<class 'rdflib.graph.Graph'>)>

In [51]:
g.serialize(destination="work.ttl", format='ttl')

<Graph identifier=Nf8a4447e118c409ab437162e76a2afcd (<class 'rdflib.graph.Graph'>)>